In [1]:
# import sys  
# from pathlib import Path
# sys.path[0] = str(Path(sys.path[0]).parent)
# print(sys.path)

In [2]:
import argparse
from collections import defaultdict
from src.utils.utils import (
    set_random_seed,
    merge_from_dict
)
import torch
from tqdm import tqdm
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import auc as auc_fn

from loguru import logger
import os
import json
import inspect
import yaml
import numpy as np
import itertools
from pathlib import Path
from src.few_shot_methods import ALL_FEW_SHOT_CLASSIFIERS
from src.detectors import ALL_DETECTORS
from src.utils.plots_and_metrics import show_all_metrics_and_plots, update_csv
from src.utils.data_fetchers import get_task_loader, get_test_features
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from src.constants import BACKBONES
from types import SimpleNamespace

In [5]:
# Variables
args = SimpleNamespace()

args.src_dataset = "mini_imagenet"
args.tgt_dataset = "mini_imagenet"
args.training = "standard"
args.model_source = "feat"
args.layers=["4_4"]

args.n_way = 5
args.n_shot = 1
args.n_query = 1
args.n_tasks = 1000
args.backbone = 'resnet12'


In [6]:
# Setup

feature_dic = defaultdict(dict)
average_train_features = {}
std_train_features = {}
for i, layer in enumerate(layers):
    features, _, average_train_features[i], std_train_features[i] = get_test_features(
        backbone, src_dataset, tgt_dataset, training, model_source, layer
    )
    for class_ in features:
        feature_dic[class_.item()][layer] = features[class_]
data_loader = get_task_loader(args, "test", tgt_dataset, n_way, n_shot,
                              n_query, n_tasks, n_workers, feature_dic)

2022-02-23 11:33:08.539 | INFO     | src.utils.data_fetchers:get_test_features:179 - Loading features from data/features/mini_imagenet/mini_imagenet/test/standard/resnet12_mini_imagenet_feat_4_4.pickle


NameError: name 'args' is not defined